In [2]:
import os
import shutil
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Activation, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from sklearn.metrics import confusion_matrix

In [3]:
source_dir='/Users/kevinkomban/Documents/Databyte Inductions/Flowers Dataset/flowers'
target='/Users/kevinkomban/Documents/Databyte Inductions/Flowers Dataset'
for split in ['train','test','valid']:
    for class_name in os.listdir(source_dir):
        os.makedirs(os.path.join(target,split,class_name),exist_ok=True)
    
for class_name in os.listdir(source_dir):
    class_path=os.path.join(source_dir,class_name)
    images=os.listdir(class_path)
    random.shuffle(images)
    n_total=len(images)
    n_train=round(n_total*0.70)
    n_test=round(n_total*0.15)
    n_valid=round(n_total*0.15)
    splits={'train':images[:n_train],'valid':images[n_train:n_train+n_valid],'test':images[n_train+n_valid:n_train+n_valid+n_test]}
    for split_names, image_list in splits.items():
        for image in image_list:
            src=os.path.join(class_path,image)
            dest=os.path.join(target,split_names,class_name,image)
            shutil.copy(src,dest)


In [4]:
train_path='/Users/kevinkomban/Documents/Databyte Inductions/Flowers Dataset/train'
test_path='/Users/kevinkomban/Documents/Databyte Inductions/Flowers Dataset/test'
valid_path='/Users/kevinkomban/Documents/Databyte Inductions/Flowers Dataset/valid'

In [8]:
train_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=train_path,target_size=(224,224), classes=['tulip','sunflower','rose','dandelion','daisy'], batch_size=10)
test_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=test_path,target_size=(224,224), classes=['tulip','sunflower','rose','dandelion','daisy'], batch_size=1,shuffle=False)
valid_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=valid_path,target_size=(224,224), classes=['tulip','sunflower','rose','dandelion','daisy'], batch_size=10)

Found 4316 images belonging to 5 classes.
Found 2292 images belonging to 5 classes.
Found 2712 images belonging to 5 classes.


In [6]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6543393221096066492
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
locality {
  bus_id: 1
}
incarnation: 7601617530019259014
physical_device_desc: "device: 0, name: METAL, pci bus id: <undefined>"
xla_global_id: -1
]


2025-07-09 21:28:03.269327: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-07-09 21:28:03.269458: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-07-09 21:28:03.269488: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
I0000 00:00:1752076683.269959  230103 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1752076683.270729  230103 pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [9]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_batches.num_classes, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_batches, epochs=10, validation_data=valid_batches)
test_loss, test_acc = model.evaluate(test_batches)
print(f"\n Test Accuracy: {test_acc:.2f}")

2292/2292 ━━━━━━━━━━━━━━━━━━━━ 22s 9ms/step - accuracy: 0.9918 - loss: 0.4556

 Test Accuracy: 0.99
